## Setup modification exercise
* Replace broad crested weir with Honma weir
* Change crest width during simulation

### Retrieve Mike1D nuget packages

In [ ]:
#r nuget:DHI.Mike.Install
// #r nuget:DHI.Mike1D.BoundaryModule
// #r nuget:DHI.Mike1D.ControlModule
// #r nuget:DHI.Mike1D.CrossSectionModule
// #r nuget:DHI.Mike1D.DataAssimilationModule
#r nuget:DHI.Mike1D.Engine
// #r nuget:DHI.Mike1D.FloodMap
#r nuget:DHI.Mike1D.Generic
// #r nuget:DHI.Mike1D.HDParameterDataAccess
#r nuget:DHI.Mike1D.Mike1DDataAccess
#r nuget:DHI.Mike1D.NetworkDataAccess
// #r nuget:DHI.Mike1D.RainfallRunoffModule
// #r nuget:DHI.Mike1D.ResultDataAccess
// #r nuget:DHI.Mike1D.SimLauncher
#r nuget:DHI.Mike1D.StructureModule

### Import the relevant libraries

In [ ]:
using System.Linq;
using DHI.Mike.Install;
using DHI.Mike1D.Engine;
using DHI.Mike1D.Generic;
using DHI.Mike1D.Mike1DDataAccess;
using DHI.Mike1D.Plugins;
using DHI.Mike1D.StructureModule;

### Setup an installation, which will be used to run MIKE 1D

In [ ]:
if (MikeImport.SetupLatest())
{
  var product = MikeImport.ActiveProduct();
  Console.WriteLine(product.InstallRoot);
}

### Specify the initial script body

In [ ]:
public class MyScript
{
  private IStructures _structures;
  private EngineNet _engineNet;

  private string _weirId = "Weir";
  private int _weirIndex;
  private double _crestWidthInitial;

  /// <summary> Method to modify MIKE 1D data. </summary>
  [Script]
  public void Initialize(Mike1DData mike1DData)
  {
    _structures = mike1DData.Network.StructureCollection.Structures;

    Console.WriteLine("Removing the weir");
    _weirIndex = _structures.FindIndex(x => x.ID == _weirId);
    var bcWeir = (BroadCrestedWeir)_structures[_weirIndex];
    _structures.RemoveAt(_weirIndex);

    Console.WriteLine("Creating Honma weir");
    var honmaWeir = CreateHonmaWeir(bcWeir);

    Console.WriteLine("Adding Honma weir to network");
    _structures.Add(honmaWeir);
  }

  private HonmaWeir CreateHonmaWeir(BroadCrestedWeir bcWeir)
  {
    _crestWidthInitial = bcWeir.WeirGeometry.LevelWidths[0].Width;

    var honmaWeir = new HonmaWeir
    {
      ID = bcWeir.ID,
      Location = bcWeir.Location,
      CrestWidth = _crestWidthInitial,
      CrestLevel = bcWeir.InvertLevelUpstream,
      WeirCoefficient = 1.0,
    };

    return honmaWeir;
  }

  /// <summary> Method to modify the controller. </summary>
  [Script]
  public void ModifyController(IMike1DController controller)
  {
    controller.ControllerEvent += ControllerOnControllerEvent;
  }

  private void ControllerOnControllerEvent(object sender, ControllerEventArgs e)
  {
    if (e.State != ControllerState.Running)
      return;

    var controller = (IMike1DController)sender;

    _engineNet = controller.EngineNet;
    _engineNet.PreTimeStepEvent += PreTimeStepEvent;
  }

  private void PreTimeStepEvent(DateTime timeN, DateTime timeNp1, int redoCount)
  {
    var honmaWeir = (HonmaWeir)_structures[_weirIndex];
    honmaWeir.CrestWidth = _crestWidthInitial * (1.0 - timeN.Minute/600.0);
    Console.WriteLine("Time: " + _engineNet.EngineTime.TimeNp1 + " Width: " + honmaWeir.CrestWidth);
  }
}


### Load the setup and test the script code

In [ ]:
string inputFileName = @".\exercise1\exercise1.m1dx";
var myScript = new MyScript();

In [ ]:
var mike1DData = Mike1DData.Create(inputFileName);
myScript.Initialize(mike1DData);